This code checks whether there is disabled parking within a 5-minute walk of green areas

### 0. Libraries

In [ ]:
import geopandas as gpd
import pandas as pd


### 1. Disabled parking
Identify disabled parkings in each isochrone

In [40]:
# Load the shapefile of disabled parkings
disab_parking = gpd.read_file("D:/Climate_Shelter_Index/parcheggi_disabili/sosta-veicoli-a-servizio-di-persone-disabili-h.shp")
# Load the shapefile of isochrones
isochrones = gpd.read_file ("D:/Climate_Shelter_Index/isocrona/5min_isochrones.shp" )

In [41]:
disab_parking

,subcat,via,civico,numero,disposti,localizzazi,geometry
0,GRATUITO,BEROALDO,4/2,1,PETTINE,BEROALDO 4/2,POINT (11.36698 44.50230)
1,GRATUITO,NICOLO DELLÃ¢â¬â¢ARCA,34,1,LINEA,NICOLO DELL'ARCA 34,POINT (11.34320 44.51212)
2,GRATUITO,VIA DEL COMMERCIO ASSOCIATO,3,5,PETTINE,VIA DEL COMMERCIO ASSOCIATO 3,POINT (11.40057 44.51677)
3,GRATUITO,VIA DEMETRIO MARTINELLI,20,1,LINEA,VIA DEMETRIO MARTINELLI 20,POINT (11.30290 44.50947)
4,GRATUITO,VIA DEL CARPENTIERE,40,3,PETTINE,VIA DEL CARPENTIERE 40,POINT (11.38893 44.50124)
...,...,...,...,...,...,...,...
1923,GRATUITO,VIA NICOLO DALLARCA,44,1,SPINA,VIA NICOLO DALLARCA 44,POINT (11.34357 44.51332)
1924,GRATUITO,MARTIRI LAGER NAZISTI,18,2,PETTINE,MARTIRI LAGER NAZISTI 18,POINT (11.30685 44.49879)
1925,GRATUITO,VIA VIZZANI,40,1,LINEA,VIA VIZZANI 40,POINT (11.36778 44.49134)
1926,GRATUITO,VIA GUINIZELLI,2,1,LINEA,VIA GUINIZELLI 2,POINT (11.35863 44.48618)


In [ ]:
results_df =gpd.sjoin(isochrones[['min_dist','nome', 'geometry']], disab_parking[['numero', 'geometry']], 
          how="left", op="intersects")

In [46]:
results_df

,nome,min_dist,numero
0,AIUOLA VIA ALTOBELLI,5,2
0,AIUOLA VIA ALTOBELLI,5,2
0,AIUOLA VIA ALTOBELLI,5,1
0,AIUOLA VIA ALTOBELLI,5,1
0,AIUOLA VIA ALTOBELLI,5,2
...,...,...,...
406,VERDE SPONDA RUZZOLA,5,1
406,VERDE SPONDA RUZZOLA,5,1
406,VERDE SPONDA RUZZOLA,5,1
407,VIALE FELSINA-VICOLO NERO,5,1


In [45]:
# Keep only the desired columns from the filtered DataFrame
results_df = results_df[['nome', 'min_dist', 'numero']]

In [ ]:
# Convert 'numero' to integers and fill missing values with 0
results_df['numero'] = results_df['numero'].fillna(0).astype(int)

In [53]:
# Count the occurrences of each 'nome' and sum 'numero'
sum_result = results_df.groupby('nome')['numero'].sum().reset_index()

In [54]:
sum_result = sum_result.rename(columns={'numero':'disab_p_5'})

In [55]:
sum_result

,nome,disab_p_5
0,AIUOLA VIA ALTOBELLI,12
1,AIUOLE DANTI,5
2,AIUOLE RAMENGHI,9
3,AIUOLE AIMO,19
4,AIUOLE COMPARTO BECCACCINO LA PIRA CAPITINI,19
...,...,...
404,VERDE CONDOMINIO VIA GALEAZZA 14 - 28,2
405,VERDE PASOLINI - ORTI (EX GIARDINO P. P. PASOL...,8
406,VERDE SPONDA RUZZOLA,7
407,VIALE FELSINA-VICOLO NERO,1


In [57]:
# Save the GeoDataFrame as a shapefile
sum_result.to_csv('D:/Climate_Shelter_Index/isocrona/disabled_parking.csv', index=False)